<h1> Reading NEXRAD Level II data from Google Cloud public datasets </h1>

This notebook demonstrates how to use PyART to visualize data from the Google Cloud public dataset.

In [ ]:
%bash
rm -rf data
mkdir data
cd data
RADAR=KIWA
YEAR=2013
MONTH=07
DAY=23
HOUR=23
gsutil cp gs://gcp-public-data-nexrad-l2/$YEAR/$MONTH/$DAY/$RADAR/*_$RADAR_${YEAR}${MONTH}${DAY}${HOUR}0000_${YEAR}${MONTH}${DAY}${HOUR}5959.tar temp.tar
tar xvf temp.tar
rm *.tar
ls

<h3> Install Py-ART </h3>

See https://github.com/ARM-DOE/pyart/wiki/Simple-Install-of-Py-ART-using-Anaconda

<h3> Plot volume scans using Py-ART within Jupyter </h3>

In [ ]:
# Based on
# http://arm-doe.github.io/pyart/dev/auto_examples/plotting/plot_nexrad_multiple_moments.html
# by Jonathan J. Helmus (jhelmus@anl.gov)
import matplotlib.pyplot as plt
import pyart
def plot_data(filename):
  radar = pyart.io.read_nexrad_archive(infilename)
  display = pyart.graph.RadarDisplay(radar)
  fig = plt.figure(figsize=(10, 10))

  ax = fig.add_subplot(221)
  display.plot('velocity', 1, ax=ax, title='Doppler Velocity',
             colorbar_label='',
             axislabels=('', 'North South distance from radar (km)'))
  display.set_limits((-300, 300), (-300, 300), ax=ax)
  ax = fig.add_subplot(222)
  display.plot('reflectivity', 0, ax=ax,
             title='Reflectivity lowest', colorbar_label='',
             axislabels=('', ''))
  display.set_limits((-300, 300), (-300, 300), ax=ax)

  ax = fig.add_subplot(223)
  display.plot('reflectivity', 1, ax=ax,
             title='Reflectivity second', colorbar_label='')
  display.set_limits((-300, 300), (-300, 300), ax=ax)

  ax = fig.add_subplot(224)
  display.plot('cross_correlation_ratio', 0, ax=ax,
             title='Correlation Coefficient', colorbar_label='',
             axislabels=('East West distance from radar (km)', ''))
  display.set_limits((-300, 300), (-300, 300), ax=ax)

  plt.show()

<h2> Plot into png </h2>

In [6]:
%writefile plot_pngs.py
import matplotlib.pyplot as plt
import pyart

def plot_data(infilename, outpng):
  radar = pyart.io.read_nexrad_archive(infilename)
  display = pyart.graph.RadarDisplay(radar)
  fig = plt.figure(figsize=(10, 10))

  ax = fig.add_subplot(221)
  display.plot('velocity', 1, ax=ax, title='Doppler Velocity',
             colorbar_label='',
             axislabels=('', 'North South distance from radar (km)'))
  display.set_limits((-300, 300), (-300, 300), ax=ax)
  ax = fig.add_subplot(222)
  display.plot('reflectivity', 0, ax=ax,
             title='Reflectivity lowest', colorbar_label='',
             axislabels=('', ''))
  display.set_limits((-300, 300), (-300, 300), ax=ax)

  ax = fig.add_subplot(223)
  display.plot('reflectivity', 1, ax=ax,
             title='Reflectivity second', colorbar_label='')
  display.set_limits((-300, 300), (-300, 300), ax=ax)

  ax = fig.add_subplot(224)
  display.plot('cross_correlation_ratio', 0, ax=ax,
             title='Correlation Coefficient', colorbar_label='',
             axislabels=('East West distance from radar (km)', ''))
  display.set_limits((-300, 300), (-300, 300), ax=ax)

  fig.savefig(outpng)
  
if __name__ == '__main__':
  import argparse
  parser = argparse.ArgumentParser(description='plot some radar data')
  parser.add_argument('nexrad', help="volume scan filename")
  parser.add_argument('png', help="output png filename")
  args = parser.parse_args()

  print "Plotting {} into {}".format(args.nexrad, args.png)
  plot_data(args.nexrad, args.png)

Overwriting plot_pngs.py


In [ ]:
%bash
python plot_pngs.py data/KIWA20130723_235451_V06.gz radarplot.png

<h2> Create animating PNG </h2>

In [ ]:
%bash
rm -rf images
mkdir images
for volumefile in $(ls data); do
    base=$(basename $volumefile)
    python plot_pngs.py data/$volumefile images/$base.png
done